## Librerias

In [1]:
from werkzeug.wrappers import Request, Response

import collections
from amigocloud import AmigoCloud

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


## Conectar a AmigoCloud

In [2]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

## Buscar registros de soca

In [3]:
semana = 13

In [5]:
query = {'query': 'select insp.canhero, replace(insp.propiedad, \'_\', \' \') propiedad, insp.user, soca.observacion, gal.s3_filename from dataset_302136 soca\
         inner join dataset_302115 insp on insp.amigo_id=soca.insp_ref_soca\
         inner join gallery_45144 gal on soca.amigo_id=gal.source_amigo_id\
         where gal.s3_filename is not null and date_part(\'week\', soca.fecha_de_registro)={sem}'.format(sem=semana)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32424/sql', query)
data = select['data']
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())
print('end')

end


In [6]:
data

[{'canhero': '3998 / FLORES LUCIANO',
  'propiedad': '863 / P-116 COOP.AGRIC.DE P.GUALBERTO VILLARROEL',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Buen control de maleza ',
  's3_filename': '45144_20230328162018049.jpg'},
 {'canhero': '3998 / FLORES LUCIANO',
  'propiedad': '802 / P-92 COOP.AGRIC.DE P.GUALBERTO VILLARROEL',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Bremura en la orilla "recomendación" paraquat 1.5L + americano 1.5 L',
  's3_filename': '45144_20230328165253056.jpg'},
 {'canhero': '1533 / CALVI PADILLA ROMAN',
  'propiedad': '104 / EL CARMEN II--CALVI',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Buen control de maleza.\nObservaciones, bremura en orilla \n',
  's3_filename': '20230331_132237.jpg'},
 {'canhero': '430 / AMURRIO QUINTANILLA FREDDY',
  'propiedad': '1252 / PARCELA 41--JUANA QUINTANILLA DE AMURRIO',
  'user': 'ingenieriaagricola2016@gmail.com',
  'observacion': 'Premura en la orilla ',
  's3_file

In [7]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/template_soca_fotos.docx")

lista_fotos_inline = []
for foto in data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({
        'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(70)), 
        'canhero': foto['canhero'],
        'propiedad': foto['propiedad'],
        'obs' : foto['observacion']
    })
    print(foto['s3_filename'])

context = {'fotos':lista_fotos_inline}

doc.render(context)
file_name = 'SOCA SEMANA - ' + str(semana)
doc.save('_' + file_name + '.docx')

45144_20230328162018049.jpg
45144_20230328165253056.jpg
20230331_132237.jpg
45144_20230327104521424.jpg
